In [305]:
import pandas as pd
import numpy as np
import ast

In [306]:
movies=pd.read_csv("tmdb_5000_movies.csv")
credits=pd.read_csv("tmdb_5000_credits.csv")
movies.shape


(4803, 20)

In [307]:
movies=movies.merge(credits,on='title')
movies.shape

(4809, 23)

In [308]:
movies=movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [309]:
movies.dropna(inplace=True)

In [310]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [311]:
def convert0(obj):
    return [obj]

In [312]:
movies['overview']=movies['overview'].apply(convert0)

In [313]:
def convert(obj):
    l=[]
    
    obj=ast.literal_eval(obj)
    for i in obj:
        l.append(i['name'])
    return l 

In [314]:
movies['genres']=movies['genres'].apply(convert)


In [315]:
movies['keywords']=movies['keywords'].apply(convert)

In [316]:
 def convert2(obj):
    l=[]
    
    obj=ast.literal_eval(obj)
    for i in obj:
        if(i['order']>2):
            break
        l.append(i['name'])
    return l 

In [317]:
movies['cast']=movies['cast'].apply(convert2)

In [318]:
def convert3(obj):
    l=[]
    obj=ast.literal_eval(obj)
    for i in obj:
        if(i['job']=='Director'):
            l.append(i['name'])
            break
    return l

In [319]:
movies['crew']=movies['crew'].apply(convert3)

In [320]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])

In [321]:
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [322]:
movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']
new_df=movies[['movie_id','title','tags']]

In [323]:
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))
new_df['tags']=new_df['tags'].apply(lambda x:x.lower())

/var/folders/vs/x4p0x4211qbgdwvzr1wgllz40000gn/T/ipykernel_4342/858905982.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))
/var/folders/vs/x4p0x4211qbgdwvzr1wgllz40000gn/T/ipykernel_4342/858905982.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:x.lower())


In [324]:
new_df.head()
new_df.isnull().sum()

movie_id    0
title       0
tags        0
dtype: int64

In [325]:
import nltk
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [326]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)
     

In [327]:
new_df['tags']=new_df['tags'].apply(stem)

/var/folders/vs/x4p0x4211qbgdwvzr1wgllz40000gn/T/ipykernel_4342/3514595201.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(stem)


In [328]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')

In [329]:
vectors=cv.fit_transform(new_df['tags']).toarray()

In [330]:
cv.get_feature_names_out().shape

(5000,)

In [340]:
from sklearn. metrics.pairwise import cosine_similarity
similarity=cosine_similarity(vectors)

In [341]:
def recommend(movie):
    movie_index=new_df[new_df['title']==movie].index[0]
    distances=similarity[movie_index]
    movies_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    for i in movies_list:
        print(new_df.iloc[i[0]].title)
    return

In [343]:
recommend('Batman Begins')

The Dark Knight
Batman
Batman
The Dark Knight Rises
10th & Wolf


In [345]:
import pickle
pickle.dump(new_df,open('movies.pkl','wb'))